<a href="https://colab.research.google.com/github/CalebEng/Ai-Project-learning/blob/master/Ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imports
import torch
import torch.nn as nn
from torch.nn import functional as F

!pip install --upgrade tiktoken
import tiktoken
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Ai work')
#torch.__version__
enc = tiktoken.encoding_for_model('gpt-3.5-turbo')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.7 MB/s eta 0:00:00
Mounted at /content/drive



------------------- Prep and loading data sets -------------------

--ONLY RUN THE NEXT IF CSV FILE CONTAINING CONVOS HAS A INDEX COLUMN--

In [ ]:
import pandas as pd
df = pd.read_csv('poems.csv')
# If you know the name of the column skip this
first_column = df.columns[0]
sec_col = df.columns[2]
trd_col = df.columns[3]
four_col = df.columns[4]

# Delete first
df = df.drop([first_column], axis=1)
df = df.drop([sec_col],axis=1)
df = df.drop([trd_col],axis = 1)
df = df.drop([four_col],axis = 1)


df.to_csv('poem.csv',index = False)


In [ ]:
df.to_csv('poems.csv',index=False)

In [ ]:
df = pd.read_csv('poem.csv')

In [ ]:
stdf = df['content'].tolist()

In [ ]:

df2 = pd.read_csv('Conversation.csv')

questions = df2['question'].tolist()
answers = df2['answer'].tolist()

temp = map(format, questions)
temp2 = map(format, answers)

questions = list(temp)
answers = list(temp2)


--EXIT--

In [ ]:
poems=""
for i in stdf:
  poems+=i
  poems+="\n\n\n"
poems=poems[:-1]

print(poems[:10000])

Let the bird of loudest lay
On the sole Arabian tree
Herald sad and trumpet be,
To whose sound chaste wings obey.

But thou shrieking harbinger,
Foul precurrer of the fiend,
Augur of the fever's end,
To this troop come thou not near.

From this session interdict
Every fowl of tyrant wing,
Save the eagle, feather'd king;
Keep the obsequy so strict.

Let the priest in surplice white,
That defunctive music can,
Be the death-divining swan,
Lest the requiem lack his right.

And thou treble-dated crow,
That thy sable gender mak'st
With the breath thou giv'st and tak'st,
'Mongst our mourners shalt thou go.

Here the anthem doth commence:
Love and constancy is dead;
Phoenix and the Turtle fled
In a mutual flame from hence.

So they lov'd, as love in twain
Had the essence but in one;
Two distincts, division none:
Number there in love was slain.

Hearts remote, yet not asunder;
Distance and no space was seen
'Twixt this Turtle and his queen:
But in them it were a wonder.

So between them love di

--Padding and turning data into tensors--

In [ ]:
df = pd.read_csv('Conversation.csv')

temp1 = df['question'].tolist()
temp2 = df['answer'].tolist()

def format(x):
  y=enc.encode(x)
  return y

qT = list(map(format, temp1))
aT = list(map(format, temp2))

for i in range(0,len(qT)):
  if len(qT[i])<25:
    while len(qT[i])<25:
      qT[i].append(482)


for i in range(0,len(aT)):
  if len(aT[i])<25:
    while len(aT[i])<25:
      aT[i].append(482)

lists = [qT,aT]
data = [val for tup in zip(*lists) for val in tup]







In [ ]:
encPoems = enc.encode(poems)



In [ ]:
tenData = torch.tensor(encPoems)

In [ ]:
tenData.type


<function Tensor.type>

In [ ]:
n = int(0.9*len(tenData))

train_data = tenData[:n]

val_data = tenData[n:]


In [ ]:
block_size = 8


In [ ]:
x = train_data[:block_size]


for t in range(block_size):
  context = x[:t+1]
  target = x[t]
  print(f"when input is {context} \nthe target is: {target}")

when input is tensor([10267]) 
the target is: 10267
when input is tensor([10267,   279]) 
the target is: 279
when input is tensor([10267,   279, 12224]) 
the target is: 12224
when input is tensor([10267,   279, 12224,   315]) 
the target is: 315
when input is tensor([10267,   279, 12224,   315, 29740]) 
the target is: 29740
when input is tensor([10267,   279, 12224,   315, 29740,  5086]) 
the target is: 5086
when input is tensor([10267,   279, 12224,   315, 29740,  5086, 11203]) 
the target is: 11203
when input is tensor([10267,   279, 12224,   315, 29740,  5086, 11203,   319]) 
the target is: 319


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 3805,   319, 59522, 27027,  1901, 97353, 20962,  1550],
        [  361,  8617,   430,  2316,   656,   339,   834, 35563],
        [  389,  1077,  1093,   264, 59380,  3304,  2675,  1518],
        [ 2967,   347,    11,   779,   606,   439,   872, 17457]])
targets:
torch.Size([4, 8])
tensor([[  319, 59522, 27027,  1901, 97353, 20962,  1550, 57150],
        [ 8617,   430,  2316,   656,   339,   834, 35563,  1909],
        [ 1077,  1093,   264, 59380,  3304,  2675,  1518,   279],
        [  347,    11,   779,   606,   439,   872, 17457,   409]])
----
when input is [3805] the target: 319
when input is [3805, 319] the target: 59522
when input is [3805, 319, 59522] the target: 27027
when input is [3805, 319, 59522, 27027] the target: 1901
when input is [3805, 319, 59522, 27027, 1901] the target: 97353
when input is [3805, 319, 59522, 27027, 1901, 97353] the target: 20962
when input is [3805, 319, 59522, 27027, 1901, 97353, 20962] the target: 1550
when inpu

In [ ]:
print(xb) # our input to the transformer

tensor([[ 3805,   319, 59522, 27027,  1901, 97353, 20962,  1550],
        [  361,  8617,   430,  2316,   656,   339,   834, 35563],
        [  389,  1077,  1093,   264, 59380,  3304,  2675,  1518],
        [ 2967,   347,    11,   779,   606,   439,   872, 17457]])


**MODEL** **TIME**

In [33]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()

        # embedding layer contains embeddings for each word
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        # linear layers to extract patterns from the corpus
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [34]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:
        context_idxs = torch.tensor([word_to_idx[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_idx[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

IndexError: ignored